In [ ]:
f = open('/Users/enn/Desktop/nlp/project1/abcnews-data.csv')
file = f.readlines()
l = []
for line in file:
    line = line.replace('\n', '')
    if 'climate' in line:
        #print(line[9:])
        
        l.append(line[9:])
print(l)

In [4]:
f = open('/Users/enn/Desktop/nlp/project1/abcnews-data.csv')
file = f.readlines()
f2 = open('2.txt', 'wb')
for line in file:
    f2.write(line)
f2.close()

TypeError: a bytes-like object is required, not 'str'

In [26]:
import requests
import bs4
import os
import datetime
import time

In [27]:
import json
f = open('/Users/enn/Desktop/nlp/project1/neg.txt')
data = f.readlines()
l = []
for item in data:
    item = item.replace('\n', '')
    item = item.replace('  ', '')
    l.append(item)
#print(l)

In [28]:
f1 = open('/Users/enn/Desktop/nlp/project1/neg_data/climate.txt')
f2 = open('/Users/enn/Desktop/nlp/project1/neg_data/non-climate.txt')
f3 = open('/Users/enn/Desktop/nlp/project1/neg_data/skeptical/total_neg.txt')
file1 = f1.readlines()
file2 = f2.readlines()
file3 = f3.readlines()
neg_data = file1 + file2 + file3
#print(neg_data)
for item in neg_data:
    item = item.replace('\n', '')
    #print(item)
print(len(neg_data))

1386


In [29]:
import json
f1 = open('/Users/enn/Desktop/nlp/project1/project-files/train.json')
f2 = open('/Users/enn/Desktop/nlp/project1/neg_data/skeptical/total_pos.txt')
file1 = json.load(f1)
file2 = f2.readlines()
#print(len(pos_data))
#print(pos_data)
pos = []
for value in file1.values():
    #print(value['text'])
    pos.append(value['text'])
    #for key in value.items():
        #if key == 'text':
            #print(value[key])
    #break
pos_data = pos + file2
for item in pos_data:
    item = item.replace('\n', '')
print(len(pos_data))

1380


In [30]:
train = neg_data + pos_data
print(len(train))

2766


In [31]:
f = open('/Users/enn/Desktop/nlp/project1/project-files/dev.json')
dev_data = json.load(f)
print(len(dev_data))
dev = []
y_dev = []
for value in dev_data.values():
    dev.append(value['text'])
    y_dev.append(value['label'])
for item in dev:
    item = item.replace('\n', '')
print(len(dev))
print(len(y_dev))

100
100
100


In [32]:
f = open('/Users/enn/Desktop/nlp/project1/project-files/test-unlabelled.json')
test_data = json.load(f)
print(len(test_data))
#print(pos_data)
test = []
for value in test_data.values():
    #print(value['text'])
    test.append(value['text'])
    #for key in value.items():
        #if key == 'text':
            #print(value[key])
    #break
for item in test:
    item = item.replace('\n', '')
print(len(test))

1410
1410


In [68]:
def getNClasses(n, label):
    return [label for i in range(n)]

y_train = getNClasses(1386, "0") + getNClasses(1380, "1")
print(len(y_train))

import random
random.seed(20)
random.shuffle(train)
random.seed(20)
random.shuffle(y_train)

2766


In [69]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words = 10000)
token.fit_on_texts(train)
#print(train)
train_seq = token.texts_to_sequences(train)
#for item in train_seq:
#print(train_seq)
dev_seq  = token.texts_to_sequences(dev)
#print(dev_seq)
x_train = sequence.pad_sequences(train_seq, maxlen = 3800)
x_dev = sequence.pad_sequences(dev_seq, maxlen = 3800)
print(x_train)

[[   0    0    0 ...  769 8004   27]
 [   0    0    0 ...   47 1275 1299]
 [   0    0    0 ...   39   51 1740]
 ...
 [   0    0    0 ...   66 9967   26]
 [   0    0    0 ...    3  142 2341]
 [   0    0    0 ...  425   49 3745]]


In [70]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
import numpy as np

model = Sequential()

model.add(Embedding(vocab_size, 256, input_length=sentence_max_len))

model.add(Dropout(0.35))

model.add(SimpleRNN(units = 16))

model.add(Dense(units = 256, activation = 'relu'))

model.add(Dropout(0.35))

model.add(Dense(units = 1, activation = 'sigmoid'))

model.summary()

###train the model
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

train_history = model.fit(np.array(x_train), np.array(y_train), batch_size = 512,
                          epochs = 10, verbose = 2,
                          validation_split = 0.2)

#results = model.evaluate(x_dev, y_dev)
#print(results)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 5000, 32)          160000    
_________________________________________________________________
dropout_23 (Dropout)         (None, 5000, 32)          0         
_________________________________________________________________
simple_rnn_12 (SimpleRNN)    (None, 16)                784       
_________________________________________________________________
dense_25 (Dense)             (None, 256)               4352      
_________________________________________________________________
dropout_24 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 257       
Total params: 165,393
Trainable params: 165,393
Non-trainable params: 0
_______________________________________________

ValueError: Error when checking input: expected embedding_15_input to have shape (5000,) but got array with shape (3800,)

In [59]:
from sklearn.metrics import f1_score

predict = model.predict_classes(x_dev)
#results = model.evaluate(x_dev, y_dev)
#print(predict)
#print(results)
print(f1_score(y_dev, predict))

predict[:10]
predict.shape
predict_classes = predict.reshape(100)
predict_classes

0.7272727272727273


array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1], dtype=int32)

In [57]:
#SentimentDict={1:'mis',0:'nonmis'}
#l = []
#from sklearn.metrics import classification_report

x_test_seq = token.texts_to_sequences(test)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 3800)
predict_result = model.predict_classes(x_test)

print(predict_result)
#print(l)
#predict[:10]
#predict.shape
#predict_classes = predict.reshape(100)
#predict_classes

[[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [1]]


In [56]:
list = predict_result.tolist()
print(len(list))

1410


In [41]:
label = []
for item in list:
    for i in item:
        label.append(i)
print(label)


[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [42]:
result = []
for item in range(1410):
    i = 'test-' + str(item)
    result.append(i)
print(result)

['test-0', 'test-1', 'test-2', 'test-3', 'test-4', 'test-5', 'test-6', 'test-7', 'test-8', 'test-9', 'test-10', 'test-11', 'test-12', 'test-13', 'test-14', 'test-15', 'test-16', 'test-17', 'test-18', 'test-19', 'test-20', 'test-21', 'test-22', 'test-23', 'test-24', 'test-25', 'test-26', 'test-27', 'test-28', 'test-29', 'test-30', 'test-31', 'test-32', 'test-33', 'test-34', 'test-35', 'test-36', 'test-37', 'test-38', 'test-39', 'test-40', 'test-41', 'test-42', 'test-43', 'test-44', 'test-45', 'test-46', 'test-47', 'test-48', 'test-49', 'test-50', 'test-51', 'test-52', 'test-53', 'test-54', 'test-55', 'test-56', 'test-57', 'test-58', 'test-59', 'test-60', 'test-61', 'test-62', 'test-63', 'test-64', 'test-65', 'test-66', 'test-67', 'test-68', 'test-69', 'test-70', 'test-71', 'test-72', 'test-73', 'test-74', 'test-75', 'test-76', 'test-77', 'test-78', 'test-79', 'test-80', 'test-81', 'test-82', 'test-83', 'test-84', 'test-85', 'test-86', 'test-87', 'test-88', 'test-89', 'test-90', 'test-91

In [44]:
re = {}
neg = {"label": 0}
pos = {"label": 1}
for i in result[0:]:
    t = int(i[-1])
    if label[t] == 1:
        re[i] = pos
    else:
        re[i] = neg
print(re)
        


{'test-0': {'label': 0}, 'test-1': {'label': 0}, 'test-2': {'label': 0}, 'test-3': {'label': 0}, 'test-4': {'label': 1}, 'test-5': {'label': 0}, 'test-6': {'label': 0}, 'test-7': {'label': 0}, 'test-8': {'label': 0}, 'test-9': {'label': 1}, 'test-10': {'label': 0}, 'test-11': {'label': 0}, 'test-12': {'label': 0}, 'test-13': {'label': 0}, 'test-14': {'label': 1}, 'test-15': {'label': 0}, 'test-16': {'label': 0}, 'test-17': {'label': 0}, 'test-18': {'label': 0}, 'test-19': {'label': 1}, 'test-20': {'label': 0}, 'test-21': {'label': 0}, 'test-22': {'label': 0}, 'test-23': {'label': 0}, 'test-24': {'label': 1}, 'test-25': {'label': 0}, 'test-26': {'label': 0}, 'test-27': {'label': 0}, 'test-28': {'label': 0}, 'test-29': {'label': 1}, 'test-30': {'label': 0}, 'test-31': {'label': 0}, 'test-32': {'label': 0}, 'test-33': {'label': 0}, 'test-34': {'label': 1}, 'test-35': {'label': 0}, 'test-36': {'label': 0}, 'test-37': {'label': 0}, 'test-38': {'label': 0}, 'test-39': {'label': 1}, 'test-40'